# Objectives
## Research Questions to Answer: 
### 1. What are the final rankings and scores for the season?
### 2. What is the most ideal candidate for current players?
### 3. Which statistics actually impact performance or outcome of games?
####      a. How do you value one position vs. another in fantasy?
####      b. Does the volume the yards thrown or ran count for an ideal candidate?
### 4. Potential Revenue based on contracts (and social media)


In [2]:
#Dependencies
import os
import urllib
import json
# import simplejson
import csv
import pandas as pd
import pytest
import base64
import requests
from pprint import pprint
from datetime import datetime
from ohmysportsfeedspy.v1_0 import API_v1_0
from ohmysportsfeedspy.v1_1 import API_v1_1
from ohmysportsfeedspy.v1_2 import API_v1_2
from ohmysportsfeedspy import MySportsFeeds
msf = MySportsFeeds(version="1.2")

In [3]:
#Authentication
msf.authenticate("mdiaz","EC1000ltd")

# NFL Schedule Function
#### (Trying to turn this into 1 api call function)

In [4]:
#Build a url path:
#Create a dictionary of information needed/intested in
#Open Necessary Files
username = "mdiaz"
password = "EC1000ltd"
baseUrl = "https://api.mysportsfeeds.com/v1.2/pull/nfl/"
Authorization = "mdiaz" + ":" + "EC1000ltd"
format_type = [".csv", ".json", ".xml"]
nfl_teams = ["ARI","ATL","BAL","BUF","CAR","CHI","CIN","CLE",
              "DAL","DEN","DET","GB","HOU","IND","JAX","KC",
              "LAC","LAR","MIA","MIN","NE","NO","NYG","NYJ",
              "OAK","PHI","PIT","SD","SEA","SF","STL","TB","TEN","WAS"]
seasons = ["2014-2015-regular/", "2015-playoff/",
         "2015-2016-regular/", "2016-playoff/",
         "2016-2017-regular/", "2017-playoff/",
         "2017-2018-regular/", "2018-playoff/"]
#API ID Parameters:
api_params = ["?gameid=",
              "?fordate="]
#Feed Paramters that DON'T USE fordates/gameid
overall_feed = [
    "active_players",
    "conference_team_standings"
    "cumulative_player_stats",
    "division_team_standings",
    "full_game_schedule",
    "latest_updates",
    "overall_team_standings",
    "player_injuries",
    "playoff_team_standings"
]
#Feed Parameters that use ForDates:
fordate_feed = [
#     "current_season",
#     "daily_dfs",
#     "daily_game_schedule",
#     "daily_player_stats",
#     "roster_players",
    "scoreboard",
#     "roster_players"
]
#Feed Parameters that use GameID:
gameid_feed = ["game_boxscore",
               "game_playbyplay",
               "game_startinglineup"]


headers= {"Authorization": "Basic " + base64.b64encode('{}:{}'.format(username,password).encode('utf-8')).decode('ascii')}
#Building a list to append through nfl_schedule function to parse the dates, away, home team
nfl_season = []
nfl_dates = []
nfl_awayTeam = []
nfl_homeTeam = []
nfl_awayScores = []
nfl_homeScores = []


In [10]:
for season in seasons:
    season_loop = season
    url2 = "https://api.mysportsfeeds.com/v1.2/pull/nfl/" + season_loop + "cumulative_player_stats" + format_type[0]
    print(url2)

https://api.mysportsfeeds.com/v1.2/pull/nfl/2014-2015-regular/cumulative_player_stats.csv
https://api.mysportsfeeds.com/v1.2/pull/nfl/2015-playoff/cumulative_player_stats.csv
https://api.mysportsfeeds.com/v1.2/pull/nfl/2015-2016-regular/cumulative_player_stats.csv
https://api.mysportsfeeds.com/v1.2/pull/nfl/2016-playoff/cumulative_player_stats.csv
https://api.mysportsfeeds.com/v1.2/pull/nfl/2016-2017-regular/cumulative_player_stats.csv
https://api.mysportsfeeds.com/v1.2/pull/nfl/2017-playoff/cumulative_player_stats.csv
https://api.mysportsfeeds.com/v1.2/pull/nfl/2017-2018-regular/cumulative_player_stats.csv
https://api.mysportsfeeds.com/v1.2/pull/nfl/2018-playoff/cumulative_player_stats.csv


In [4]:
#NFL Functions for full game, active players, standings, injuries(incomplete), latest updates
# nfl_dateDict
# 1st i get the schedule
# for each schedule/sesaon i get the date, away & home team
# the function for_date needs only the date
# the function game_id needs all 3 to run
def nfl_schedule():
    
#     for season in seasons:
        #season_loo4p = season
        #print(season_loop)
        #season_l = season_loop   
        url2 = "https://api.mysportsfeeds.com/v1.2/pull/nfl/" + "2014-2015-regular/" + "full_game_schedule" + format_type[1]
#         print(url2)
        nflsch = requests.get(url2, headers= headers).json()
     
#         print(x)
#         print(json.dumps(nflsch, indent=4, sort_keys=True))
        games_list = nflsch["fullgameschedule"]["gameentry"]
        for game in games_list:
            dateOfCurrentGame = game["date"].replace("-", "")
            dateOfCurrentGame_201409 = dateOfCurrentGame
            dateOfCurrentGame_201410 = dateOfCurrentGame
            #dateFormatted = dateOfCurrentGame.replace("-", "")
            awayTeamCurrentGame = game["awayTeam"]["Abbreviation"]
            homeTeamCurrentGame = game["homeTeam"]["Abbreviation"]
#           print(dateOfCurrentGame  + "-" + awayTeamCurrentGame + "-" +  homeTeamCurrentGame)
            nfl_season.append("2014-2015-regular/")
            nfl_dates.append(dateOfCurrentGame)
            nfl_awayTeam.append(awayTeamCurrentGame)
            nfl_homeTeam.append(homeTeamCurrentGame)
#            
#             print(nfl_dates)
            
#             for types in gameid_feed:
#                 gameidURL = baseUrl + "2014-regular/" + types + format_type[1] + api_params[0] + dateOfCurrentGame + "-" + awayTeamCurrentGame + "-" + homeTeamCurrentGame
#                 print(gameidURL)
#                 nfl_boxscores = requests.get(gameidURL, headers=headers).json()
#     #                   print(nfl_boxscores.json())
              
#                 print(json.dumps(nfl_boxscores, indent=4, sort_keys=True))
                
#             for types in fordate_feed:
            if dateOfCurrentGame_201409 <= "20140929":
                fordateURL = baseUrl + "2014-regular/" + "scoreboard" + format_type[1] + api_params[1] + dateOfCurrentGame
                print(fordateURL)
                nfl_scoreboard = requests.get(fordateURL, headers=headers).json()
    #                 print(nfl_scoreboard.json())
    #                 y = nfl_scoreboard.json()
    #                 print(json.dumps(y, indent=4, sort_keys=True))
#             elif dateOfCurrentGame_201409 >= "20140929":
    #                 print(nfl_scoreboard.json())
    #                 y = nfl_scoreboard.json()
    #                 print(json.dumps(y, indent=4, sort_keys=True))
            elif dateOfCurrentGame_201410 == "20141030":
                gameScore = nfl_scoreboard["scoreboard"]["gameScore"]
                for scores in gameScore:
                    awayScores = scores["awayScore"]
                    homeScores = scores["homeScore"]
            
                    nfl_awayScores.append(awayScores)
                    nfl_homeScores.append(homeScores)
                    break
    #                 nfl_df["Season"] = [nfl_season]
    #                 nfl_df["Dates"] = [nfl_dates]
    #                             "Away Team": nfl_awayTeam,
    #                             "Home Team": nfl_homeTeam

                

In [5]:
#Trying to print game box scores within schedule
nfl_schedule()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
print(len(nfl_season))
print(len(nfl_dates))
print(len(nfl_awayTeam))
print(len(nfl_homeTeam))
print(len(nfl_awayScores))
print(len(nfl_homeScores))

In [ ]:
nfl_df = pd.DataFrame({
                            "Season": nfl_season,
                            "Dates": nfl_dates,
                            "Away Team": nfl_awayTeam,
                            "Home Team": nfl_homeTeam,
                            "Away Team Scores": nfl_awayScores,
                            "Home Team Scores": nfl_homeScores

})

#After Season, add: scores, 
nfl_df.head(2)

#### 1. What are the final rankings and scores for the season?

In [ ]:
url2 = "https://api.mysportsfeeds.com/v1.2/pull/nfl/" + "2014-2015-regular/" + "full_game_schedule" + format_type[1]
#         print(url2)
nflsch = requests.get(url2, headers= headers)
#       print(nflsch.json())
z = json.load(nflsch)
# print(json.dumps(x, indent=4, sort_keys=True))
y = csv.writer(z)
for item in z:
    values = [x.encode("utf8") for x in item["fields"].values() ]
    y.writerow([item["pk"], item["model"]] + values)